In [5]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from matplotlib.pyplot import hist, show
from collections import Counter
import urllib
import codecs
from nltk.stem import WordNetLemmatizer

## Czytania danego dnia

In [141]:
#downloading the webpage
import requests
import datetime

now = datetime.datetime.now()
year = str(now.year)
month = str(now.month)
if len(month) == 1:
    month = "0" + month
day = str(now.day)
ymd = year+month+day

    
#address = "http://mateusz.pl/czytania/" + year + "/" + ymd + ".html"
address = "http://mateusz.pl/czytania/2018/20180909.html"
page = requests.get(address)

In [172]:
#parsing a page
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [185]:
#print(soup.prettify()) 
#Exception handling for more readings
try:
    section = list(soup.children)[9]
    section_content = section.find_all('p')
except:
    section = soup.find_all('section')[0]
    section_content =section.find_all('p')

In [186]:
len(section_content)

8

In [187]:
pierwsze_czytanie = section_content[0]

In [188]:
ewangelia = section_content[len(section_content)-1]

### Lemmatyzacja (lemmatisation) 
Algorytm znajdowania lemmy, formy podstawowej wyrazu w obszarze części mowy którą reprezentuje. Inaczej: wyszukiwanie formy kanonicznej leksemu. W komputerowej analizie języka naturalnego lemmatyzacja jest operacją bardziej precyzyjną niż stemmatyzacja. 

### Stemmatyzacja (stemming) 
Sprowadzenie wyrazu reprezentującego dowolną część mowy do stemu, rdzenia postaci źródłowej, najczęściej rzeczownika, bądź niekiedy nawet sekwencji znaków nie mającej samodzielnego znaczenia w języku naturalnym. 

In [7]:
polish_stops = codecs.open("polishStopWords",'r','utf-8')

In [8]:
ps = polish_stops.read().split('\r\n')

In [9]:
milosc = codecs.open("milosc",'r','utf-8')

In [10]:
milujcie = codecs.open("milujcie",'r','utf-8')

In [11]:
milujcie_sent = milujcie.read().split('\r\n')

In [12]:
song = ""
for sentence in milujcie_sent:
    song = song + sentence

In [13]:
text = milosc.read()
#text = song

In [14]:
tokens = word_tokenize(text)
lower_tokens = [t.lower() for t in tokens]

### Lemmatization

In [15]:
# Retain alphabetic words: alpha_only
alpha_only = [t for t in lower_tokens if t.isalpha()]

In [16]:
# Remove all stop words: no_stops
no_stops = [t for t in alpha_only if t not in ps]

In [17]:
#bag of words
bow = Counter(no_stops)

In [18]:
bow.most_common(20)

[('czyńcie', 3),
 ('należy', 3),
 ('wdzięczność', 3),
 ('grzesznicy', 3),
 ('będziecie', 3),
 ('miłujcie', 2),
 ('waszych', 2),
 ('nieprzyjaciół', 2),
 ('zwrotu', 2),
 ('miłują', 2),
 ('jakaż', 2),
 ('czynią', 2),
 ('samo', 2),
 ('powiadam', 1),
 ('słuchacie', 1),
 ('nienawidzą', 1),
 ('błogosławcie', 1),
 ('przeklinają', 1),
 ('módlcie', 1),
 ('oczerniają', 1)]

In [19]:
milosc.close

<function BufferedReader.close>

In [20]:
milujcie.close

<function BufferedReader.close>